## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Batemans Bay,AU,2021-01-23 19:39:37,-35.7167,150.1833,66.00,94,10,1.01,clear sky
1,1,New Norfolk,AU,2021-01-23 19:39:57,-42.7826,147.0587,55.99,74,5,2.53,clear sky
2,2,Cape Town,ZA,2021-01-23 19:38:58,-33.9258,18.4232,68.00,94,75,14.97,light rain
3,3,Sept-Iles,CA,2021-01-23 19:39:00,50.2001,-66.3821,21.00,79,100,10.36,light snow
4,4,Butaritari,KI,2021-01-23 19:39:15,3.0707,172.7902,79.45,81,95,11.86,overcast clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 75


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
potential_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
potential_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Batemans Bay,AU,2021-01-23 19:39:37,-35.7167,150.1833,66.00,94,10,1.01,clear sky
1,1,New Norfolk,AU,2021-01-23 19:39:57,-42.7826,147.0587,55.99,74,5,2.53,clear sky
2,2,Cape Town,ZA,2021-01-23 19:38:58,-33.9258,18.4232,68.00,94,75,14.97,light rain
5,5,Salalah,OM,2021-01-23 19:39:58,17.0151,54.0924,73.40,23,0,10.36,clear sky
7,7,Ushuaia,AR,2021-01-23 19:39:28,-54.8000,-68.3000,60.80,41,75,10.58,light rain


In [14]:
# 4a. Determine if there are any empty rows.
potential_cities_df.isnull().count()

City_ID                186
City                   186
Country                186
Date                   186
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Weather Description    186
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = potential_cities_df.dropna()

In [66]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Batemans Bay,AU,66.00,clear sky,-35.7167,150.1833,
1,New Norfolk,AU,55.99,clear sky,-42.7826,147.0587,
2,Cape Town,ZA,68.00,light rain,-33.9258,18.4232,
5,Salalah,OM,73.40,clear sky,17.0151,54.0924,
7,Ushuaia,AR,60.80,light rain,-54.8000,-68.3000,
10,Hobart,AU,57.00,few clouds,-42.8794,147.3294,
12,Port Alfred,ZA,70.00,overcast clouds,-33.5906,26.8910,
14,Rawson,AR,68.90,clear sky,-43.3002,-65.1023,
15,Port Elizabeth,ZA,68.00,broken clouds,-33.9180,25.5701,
19,Matara,LK,72.82,clear sky,5.9485,80.5353,


In [67]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [69]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()


In [70]:
# 8a. Create the output File (CSV)
output_data_file = 'Vacation_Search/WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))